In [ ]:
pip install scrapy

     |████████████████████████████████| 245kB 4.9MB/s 
     |████████████████████████████████| 3.1MB 6.0MB/s 
     |████████████████████████████████| 3.2MB 24.4MB/s 
     |████████████████████████████████| 3.2MB 53.0MB/s 
     |████████████████████████████████| 245kB 37.3MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=ef0d9ec56034db8941bff25c879fa307c5a9fd1acfac239ee984219e5c514dd1
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=61a4ed91f48b1e3a0c06d36dad9f7e2f2137bc70663540ee53fd976319702b66
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatche

In [ ]:
# Scraping
import scrapy
#from scrapy import Selector
from requests import get

# Parsing
from urllib.parse import urlencode
from urllib.parse import urlparse

from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

In [ ]:
!scrapy startproject WebCrawler

New Scrapy project 'WebCrawler', using template directory '/usr/local/lib/python3.6/dist-packages/scrapy/templates/project', created in:
    /content/WebCrawler

You can start your first spider with:
    cd WebCrawler
    scrapy genspider example example.com


In [ ]:
import string
import scrapy
from scrapy import Request
​
class PubMedSpider(scrapy.Spider):
​
    name = "PubMed"
    start_urls = ['www.pubmed.ncbi.nlm.nih.gov']
​
    def parse(self, response):
        xp = "//div[@id='horiznav_nav']//li/a/@href"
        return (Request(url, callback=self.parse_manga_list_page) for url in response.xpath(xp).extract())
​
    def parse_manga_list_page(self, response):
        for tr_sel in response.css('div.js-categories-seasonal tr ~ tr'):
            yield {
                "title":  tr_sel.css('a[id] strong::text').extract_first().strip(),
                "synopsis": tr_sel.css("div.pt4::text").extract_first(),
                "type_": tr_sel.css('td:nth-child(3)::text').extract_first().strip(),
                "episodes": tr_sel.css('td:nth-child(4)::text').extract_first().strip(), 
                "rating": tr_sel.css('td:nth-child(5)::text').extract_first().strip()
            }
​
        next_urls = response.xpath("//div[@class='spaceit']//a/@href").extract()
        for next_url in next_urls:
            yield Request(response.urljoin(next_url), callback=self.parse_anime_list_page)

In [ ]:
import scrapy


class PageSpider(scrapy.Spider):
    name = 'page'
    allowed_domains = ['www.pubmed.ncbi.nlm.nih.gov']
    start_urls = ['https://pubmed.ncbi.nlm.nih.gov/']

    rules = (
        Rule(LinkExtractor(allow=(), restrict_css=('.pageNextPrev',)),
             callback="parse_item",
             follow=True),)

    def parse_item(self, response):
        print('Processing..' + response.url)

In [ ]:
!scrapy crawl --nolog WebCrawler

Scrapy 2.4.1 - no active project

Unknown command: crawl

Use "scrapy" to see available commands
